<a href="https://colab.research.google.com/github/tina043c/ML/blob/master/CNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [0]:
DIR_OUT = './'
batch_size = 128
num_classes = 10
epochs = 10


In [0]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [17]:
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')



60000 train samples
10000 test samples


In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [19]:
model_name = 'CNN1'
model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dense(512,activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation='softmax'))
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 932,362
Trainable params: 932,362
Non-trainable params: 0
________________________________________________

In [20]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

train_history= model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2,
                    shuffle=True)

score = model.evaluate(x_test, y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
375/375 [==============================] - 2s 4ms/step - loss: 0.5381 - accuracy: 0.8547 - val_loss: 0.2267 - val_accuracy: 0.9337
Epoch 2/10
375/375 [==============================] - 2s 4ms/step - loss: 0.2304 - accuracy: 0.9329 - val_loss: 0.1632 - val_accuracy: 0.9528
Epoch 3/10
375/375 [==============================] - 2s 5ms/step - loss: 0.1694 - accuracy: 0.9488 - val_loss: 0.1355 - val_accuracy: 0.9607
Epoch 4/10
375/375 [==============================] - 2s 5ms/step - loss: 0.1327 - accuracy: 0.9600 - val_loss: 0.1132 - val_accuracy: 0.9663
Epoch 5/10
375/375 [==============================] - 2s 4ms/step - loss: 0.1098 - accuracy: 0.9668 - val_loss: 0.1026 - val_accuracy: 0.9689
Epoch 6/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0921 - accuracy: 0.9727 - val_loss: 0.0984 - val_accuracy: 0.9707
Epoch 7/10
375/375 [==============================] - 2s 4ms/step - loss: 0.0783 - accuracy: 0.9764 - val_loss: 0.0883 - val_accuracy: 0.9733
Epoch 

In [12]:



from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model

history = model.fit(x_test, y_test, validation_split=0.3, epochs=10, verbose=0)

train_history= model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)


score = model.evaluate(x_test, y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)


print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('Test f1_score:', f1_score)
print('Test precision:', precision)
print('Test recall:', recall)


Epoch 1/10
375/375 [==============================] - 2s 6ms/step - loss: 0.0031 - acc: 0.9951 - f1_m: 0.9953 - precision_m: 0.9957 - recall_m: 0.9948 - val_loss: 0.0152 - val_acc: 0.9789 - val_f1_m: 0.9793 - val_precision_m: 0.9801 - val_recall_m: 0.9784
Epoch 2/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0029 - acc: 0.9952 - f1_m: 0.9953 - precision_m: 0.9958 - recall_m: 0.9949 - val_loss: 0.0152 - val_acc: 0.9793 - val_f1_m: 0.9794 - val_precision_m: 0.9798 - val_recall_m: 0.9789
Epoch 3/10
375/375 [==============================] - 2s 6ms/step - loss: 0.0026 - acc: 0.9959 - f1_m: 0.9960 - precision_m: 0.9963 - recall_m: 0.9956 - val_loss: 0.0155 - val_acc: 0.9793 - val_f1_m: 0.9793 - val_precision_m: 0.9801 - val_recall_m: 0.9785
Epoch 4/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0024 - acc: 0.9961 - f1_m: 0.9960 - precision_m: 0.9963 - recall_m: 0.9957 - val_loss: 0.0155 - val_acc: 0.9797 - val_f1_m: 0.9800 - val_precision_m: 0.9807

KeyboardInterrupt: ignored

In [11]:
import numpy as np
import matplotlib.pyplot as plt

def show_train_history(train_type, test_type):
# Get training and test loss histories
    plt.plot(train_history.history[train_type]) 
    plt.plot(train_history.history[test_type])
    plt.title('Train History')
    if train_type == 'accuracy':
        plt.ylabel('Accuracy')
    else:
        plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train','test'],loc='upper left')
    plt.show()
show_train_history('accuracy','val_accuracy')
show_train_history('loss','val_loss')

KeyError: ignored

In [0]:
# Save Keras model
def savemodel(model, model_params_dir,name):
    json_string = model.to_json()
    # Save model architecture in JSON file
    open(model_params_dir + name + '_arch.json', 'w').write(json_string)
    # Save weights as HDF5
    model.save_weights(model_params_dir + name + '_weights.h5')
    print("Saved model to disk")

In [0]:
savemodel(model, DIR_OUT, model_name)

In [0]:
# Load Keras model
def loadmodel(model_params_dir,name): 
    # Load model architecture from JSON file
    model = keras.models.model_from_json(open(model_params_dir + name + '_arch.json').read())
    # Load model weights from HDF5 file
    model.load_weights(model_params_dir + name + '_weights.h5')
    print("Loaded model from disk")
    return model

In [0]:
# Load weights only
def loadmodelweights(model,model_params_dir,name_weightfile): 
    # Load model weights from HDF5 file
    model.load_weights(model_params_dir + name_weightfile+'.h5')
    print("Loaded model weights from disk")
    return model

In [0]:
loadmodelweights(model, DIR_OUT, model_name + '_weights')